In [1]:
from shapely.geometry import Point, Polygon
import pandas as pd
import osmnx as ox

def extract_osm_data(place_name,output_file, level_map=None):
    
    # Download the OpenStreetMap data for the area
    pois = ox.geometries_from_place(place_name, tags={'amenity':True, 'building':True, 'landuse':True})

    # Convert the data to a pandas DataFrame
    df = pd.DataFrame(pois, columns=['name', 'addr:street', 'addr:city', 'addr:postcode', 'employees','amenity','building', 'landuse', 'geometry'])
    
     # Merge the 'amenity', 'building', and 'landuse' columns into a single 'type' column
    df['type'] = df['amenity'].fillna(df['building']).fillna(df['landuse'])

    # Map each point of interest to a level based on the 'amenity' tag, if a level_map is provided
    if level_map:
        df['level'] = df['type'].map(level_map)

    # Filter out any rows where the 'level' column is NaN
        df = df.dropna(subset=['level'])

    # Apply the 'extract_coordinates' function to create separate 'x' and 'y' columns
    def extract_coordinates(row):
        geom = row['geometry']
        if isinstance(geom, Point):
            x, y = geom.coords[0]
        elif isinstance(geom, Polygon):
            centroid = geom.centroid
            x, y = centroid.x, centroid.y
        else:
            raise ValueError(f"Unsupported geometry type: {type(geom)}")
        return (x or None, y or None)

    df['x'], df['y'] = zip(*df.apply(extract_coordinates, axis=1))

    # Save the resulting DataFrame to a CSV file
    df.to_csv(output_file, index=False)

    return df

In [2]:
level_map_business={
    'commercial':'commercial',
    'industrial': 'industrial',
    'office': 'office',
    'retail': 'retail',
    'coworking_space': 'coworking_space',
    'conference_centre': 'conference_centre',
    'courthouse': 'courthouse',
    'public_building': 'public_building'}

level_map_education={
    'school':'school',
    'university':'university'}

level_map_residential={
    'apartments': 'apartments',
    'house':'house',
    'villa':'villa',
    'residential':'residential'}

In [4]:
df_work =extract_osm_data("Casablanca, Morocco", 'work.csv', level_map=level_map_business)

In [6]:
df_education = extract_osm_data("Casablanca, Morocco" , 'education.csv', level_map=level_map_education)

In [7]:
df_home= extract_osm_data("Casablanca, Morocco", 'home.csv', level_map=level_map_residential)